<a href="https://colab.research.google.com/github/Patortiz26/IN6531-Miner-a-de-datos-2021-2/blob/main/Laboratorio6_Enunciado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo:

- Patricio Ortiz


### **Link de repositorio de GitHub:** https://github.com/Patortiz26/Laboratorio-de-Programaci-n`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 12/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import os
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
# !pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/My Drive/Colab Notebooks/MDS7202 - Laboratorio de programación/Lab 6'
    os.chdir(path)
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<p align="center"><iframe width="560" height="315" src="https://www.youtube.com/embed/MW4d8_LLYp8?rel=0&amp;controls=0&amp;showinfo=0" frameborder="1" allowfullscreen></iframe></p>

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)

df_retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [4]:
#PRIMERO TRABAJAMOS UN POCO LA BASE ORIGINAL

#Leemos la columna InvoiceDate como datetime y luego nos quedamos con la fecha (los calculos se hace sobre la diferencia en diías)
df_retail.InvoiceDate = pd.to_datetime(df_retail.InvoiceDate).dt.date

#Agregamos una columna con el precio total de la observacion (P*Q)
df_retail['Total Price'] = df_retail.Quantity * df_retail.Price

#Agregamos la columna deltaDate que marca la diferencia con el día actual
now = df_retail.InvoiceDate.max() + datetime.timedelta(1) #fecha maxima mas un día
df_retail['deltaDate'] = now - df_retail.InvoiceDate      #diferencia entre dias
df_retail['deltaDate'] = df_retail['deltaDate'].dt.days   #dejamos el valor entero de deltadate

df_retail.head(2)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total Price,deltaDate
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom,83.4,374
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,81.0,374


In [5]:
def custom_features(dataframe_in):
  #Primero agrupamos las facturas
  group = dataframe_in.groupby('Invoice')     #Se agrupa el dataframe inicial por la llave 'Invoice'
  df_ret= group.agg({'Customer ID': 'first',  #Nos quedamos el primer customer id (nos podriamos quedar con cualquiera)
                      'Total Price': 'sum',   #Suma del monto total gastado (por factura)
                      'InvoiceDate': 'first', #Se guarda la fecha de la factura
                      'deltaDate': 'first'})  #Idem para la columna deltaDate
  df_ret = df_ret.reset_index()               #Reiniciamos el indice para tener la columna invoice

  #Luego se agrupa a partir del dataframe anterior por cada cliente
  group = df_ret.groupby('Customer ID')                           #Se agrupa por llave "Customer ID"

  df_fin = group.agg({'InvoiceDate': lambda x: x.max() - x.min(), #Length: Diferencia entre ultima compra y la primera.
                    'deltaDate': ['mean',                         #Recency: Distancia promedio entre fecha final(hoy) y cada visita
                                  'std'] ,                        #Periodicity: Desv. estandar de la diferencia entre visitas
                    'Invoice': 'count',                           #Frecuency: Número de visitas del cliente
                    'Total Price': 'mean',                        #Monetary: Cantidad gastada por visita
                    })                         

  df_fin.columns = ['Length', 'Recency','Periodicity','Frecuency','Monetary'] #Cambio de nombre en las columnas
  df_fin['Length'] = df_fin['Length'].dt.days                     #Nos quedamos solo con el valor numerico de la columna
  df_fin['Recency'] = np.round(df_fin['Recency'])                 #Redondeamos los valores de recency
  df_fin['Monetary'] = np.round(df_fin['Monetary'])               #Redondeamos los valores de monetary
  df_fin['Periodicity'] = np.round(df_fin['Periodicity'])         #Redondeamos los valores de periodicity
  df_fin = df_fin.dropna(how='any')                               #Los clientes con solo una compra arrojan valor na en periodicity, asi que se elimina la observacion.

  df_fin = df_fin[['Length', 'Recency','Frecuency','Monetary','Periodicity']] #ordenamos las columnas según enunciado

  return df_fin

In [6]:
#Probamos la función, para comprobar que se obtiene resultado esperado
custom_features(df_retail).head()

,Length,Recency,Frecuency,Monetary,Periodicity
Customer ID,,,,,
12346.0,196,325.0,11,34.0,58.0
12347.0,37,22.0,2,662.0,26.0
12349.0,182,158.0,3,890.0,100.0
12352.0,17,20.0,2,172.0,12.0
12356.0,44,35.0,3,1187.0,23.0


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [7]:
class MinMax(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
    #Obtenemos los valores maximos y minimos de la columna a modificar
    self.Max = X.max()
    self.Min = X.min()
    return self

  def transform(self, X):
    #Transformamos la variable según definicion
    X = (X - self.Min) / (self.Max - self.Min)
    return X

### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [8]:
#Creamos Pipeline
pipe_1 = Pipeline(steps=[('Transformador', FunctionTransformer(custom_features)), #Primero se transforma el dataframe original "df_retail" según indicaciones de P1
                       ('Escalador', ColumnTransformer(transformers=[('MinMax', MinMax(),['Length', 'Recency', 'Frecuency', 'Monetary', 'Periodicity'])])), #Luego escalamos todas las columnas
                       ('Extractor', TSNE())]) #Finalmente se aplica TSNE sobre los datos escalados

#Creamos un dataframe con los datos generados por el pipeline
df_2d = pd.DataFrame(data = pipe_1.fit_transform(df_retail),   #Se ejecuta el pipeline y se ingresa el resultado como la data
                     columns = ['Dimension_x','Dimension_y'])  #Se nombran las columnas

fig = px.scatter(df_2d, x="Dimension_x", y="Dimension_y")     #Creamos el objeto a plotear
fig.show() #Ploteamos

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

Lo primero que se debe realizar es crear una función que estará dentro del Pipeline que hará el gráfico del método del codo.

In [9]:
def elbow_method(dataframe_in):
  #Armamos un dataframe donde se guardará la inertia_ y el número de clusters para un rango de entre 1 y 20 clusters
  inertias = pd.DataFrame(data =[[i, KMeans(n_clusters=i, random_state=0).fit(dataframe_in).inertia_]for i in range(1, 21)], #Se ejecuta un for sobre el numero de clusters y se guarda la inertia en una lista
                          columns = ["n° clusters", "inertia"] ) #Se nombran las columnas

  fig = px.line(inertias, x="n° clusters", y="inertia", title="Método del Codo con K-Means") #Se crea el objeto a ploteaar con los puntos "Numero de Clusters" e "Inertia"
  fig.show() #Mostramos el gráfico

In [10]:
#Actualizamos el pipeline
pipe_2 = Pipeline(steps=[('Transformador', FunctionTransformer(custom_features)), #Primero se transforma el dataframe original "df_retail" según indicaciones de P1
                       ('Escalador', ColumnTransformer(transformers=[('MinMax', MinMax(),['Length', 'Recency', 'Frecuency', 'Monetary', 'Periodicity'])])), #Luego escalamos todas las columnas
                       ('Plot Elbow', FunctionTransformer(elbow_method))])  #Ahora ejecutamos la función previamente definida

#Ejecutamos
pipe_2.fit_transform(df_retail) 

Se elige utilizar n_clusters = 4. Pues este es el valor con el cual se genera un codo al partir del cual la pendiente de la curva se mantiene "plana".

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [15]:
#Actualizamos el pipeline
pipe_3 = Pipeline(steps=[('Transformador', FunctionTransformer(custom_features)), #Primero se transforma el dataframe original "df_retail" según indicaciones de P1
                       ('Escalador', ColumnTransformer(transformers=[('MinMax', MinMax(),['Length', 'Recency', 'Frecuency', 'Monetary', 'Periodicity'])])), #Luego escalamos todas las columnas
                       ('KMeans', KMeans(n_clusters=4, random_state=0))])  #Ahora realizamos el cluster con n=4

pipe_3.fit(df_retail)
data_transformed = pd.DataFrame(data = pipe_3['Transformador'].transform(df_retail),
                                columns = ['Length', 'Recency','Frecuency','Monetary','Periodicity'])

data_transformed = data_transformed.reset_index(drop=False)

data_labels = pd.DataFrame(data = pipe_3['KMeans'].labels_,
                           columns = ['Cluster'])

data = pd.concat([data_transformed,data_labels],axis=1)

data = data.groupby('Cluster').agg({'Length': ['count','mean'], 
                             'Recency':'mean',
                             'Frecuency':'mean',
                             'Monetary':'mean',
                             'Periodicity':'mean'})

data.columns = ['Count','Length', 'Recency','Frecuency','Monetary','Periodicity']
data

,Count,Length,Recency,Frecuency,Monetary,Periodicity
Cluster,,,,,,
0,924,199.104978,151.720779,4.704545,373.521645,96.650433
1,598,53.240803,67.857860,3.091973,379.687291,29.130435
2,1038,323.588632,184.209056,10.162813,411.710019,135.193642
3,333,82.615616,261.528529,3.150150,381.615616,46.915916


In [14]:
data_labels

,Cluster
0,3
1,1
2,0
3,1
4,1
...,...
2888,2
2889,1
2890,2
2891,2


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [12]:
df_2d['Label'] = pipe_3['KMeans'].labels_

fig = px.scatter(df_2d, x="Dimension_x", y="Dimension_y", color='Label')     #Creamos el objeto a plotear
fig.show() #Ploteamos

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>